In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
import dash
from dash import dcc, html
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = pd.read_csv('CAERS_ASCII_2004_2017Q2.csv')
df.head()

,RA_Report #,RA_CAERS Created Date,AEC_Event Start Date,PRI_Product Role,PRI_Reported Brand/Product Name,PRI_FDA Industry Code,PRI_FDA Industry Name,CI_Age at Adverse Event,CI_Age Unit,CI_Gender,AEC_One Row Outcomes,SYM_One Row Coded Symptoms
0,65325,01-01-2004,08-04-2003,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL..."
1,65325,01-01-2004,08-04-2003,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL..."
2,65333,01-01-2004,NaN,Suspect,KROGER CLASSIC CREAM-DE-MINT CANDY MINT CHIP I...,13,Ice Cream Prod,NaN,Not Available,Female,VISITED AN ER,"NAUSEA, DYSGEUSIA, DIARRHOEA"
3,65335,01-01-2004,11/24/2003,Suspect,ENFAMIL LIPIL BABY FORMULA,40,Baby Food Prod,3.0,Month(s),Not Available,NON-SERIOUS INJURIES/ ILLNESS,"GASTROINTESTINAL DISORDER, VOMITING"
4,65336,01-01-2004,NaN,Suspect,ENFIMIL LIPIL BABY FORMULA,40,Baby Food Prod,NaN,Not Available,Not Available,VISITED A HEALTH CARE PROVIDER,"GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION"


In [4]:
temp_df = df.copy()

# Q1

In [5]:
temp_df['AEC_Event Start Date'] = pd.to_datetime(temp_df['AEC_Event Start Date'])

In [6]:
temp_df['CI_Gender'].unique()

array(['Female', 'Not Available', 'Male', 'Unknown', 'Not Reported'],
      dtype=object)

In [7]:
for name in temp_df['CI_Gender'].unique():
    if(name == 'Female'):
        temp_df['CI_Gender'] = temp_df['CI_Gender'].replace(name,0)
    elif(name == 'Male'):
        temp_df['CI_Gender'] = temp_df['CI_Gender'].replace(name,1)
    else:
        temp_df['CI_Gender'] = temp_df['CI_Gender'].replace(name,2)
temp_df = temp_df.replace(' ',np.nan)

In [8]:
temp_df['CI_Gender'].unique()

array([0, 2, 1], dtype=int64)

In [9]:
# df['CI_Gender'] = df['CI_Gender'].replace('Female',0)
# df['CI_Gender'] = df['CI_Gender'].replace('Male',1)



In [10]:
temp_df['AEC_Event Start Date'].replace('', np.nan, inplace=True)
temp_df.dropna(subset=['AEC_Event Start Date'], inplace=True)
temp_df.shape

(53653, 12)

In [11]:
data = temp_df[['AEC_Event Start Date','CI_Gender']]

In [12]:
data.to_csv('q1.csv',index=False)

In [13]:
dt = pd.read_csv('q1.csv')

In [14]:
years = []
for i in range(0,dt['AEC_Event Start Date'].shape[0]):
    date = pd.to_datetime(dt['AEC_Event Start Date'][i])
    date = str(date).split()[0]
    year = pd.to_numeric(date.split('-')[0])
    years.append(year)

In [15]:
dt['years'] = years

In [16]:
dt.head(2)

,AEC_Event Start Date,CI_Gender,years
0,2003-08-04,0,2003
1,2003-08-04,0,2003


In [17]:
data_count_by_year = dt.groupby('years')['CI_Gender'].count()

In [18]:
data_count_by_year = data_count_by_year.to_dict()

In [19]:
# dt = dt[(dt['CI_Gender'].str.contains('0', na=False)) | (dt['CI_Gender'].str.contains('1', na=False))]
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53653 entries, 0 to 53652
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   AEC_Event Start Date  53653 non-null  object
 1   CI_Gender             53653 non-null  int64 
 2   years                 53653 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [20]:
dt['CI_Gender'] = pd.to_numeric(dt['CI_Gender'])
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53653 entries, 0 to 53652
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   AEC_Event Start Date  53653 non-null  object
 1   CI_Gender             53653 non-null  int64 
 2   years                 53653 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [21]:
dt.index = dt['years']

In [22]:
dt_zeros = dt.where(dt['CI_Gender'] == 0)
dt_ones = dt.where(dt['CI_Gender'] == 1)
dt_unknown = dt.where(dt['CI_Gender'] == 2)

In [23]:
dt_zeros = dt_zeros.dropna()
dt_ones = dt_ones.dropna()
dt_unknown = dt_unknown.dropna()

In [24]:
dt_zeros = dt_zeros.groupby(dt_zeros.index)['CI_Gender'].count()
dt_ones = dt_ones.groupby(dt_ones.index)['CI_Gender'].count()
dt_unknown = dt_unknown.groupby(dt_unknown.index)['CI_Gender'].count()

In [25]:
dt_zeros_female = dt_zeros.to_dict()
dt_ones_male = dt_ones.to_dict()
dt_twos_unknown = dt_unknown.to_dict()

In [26]:
years = []
total_entries = []
male = []
female = []
unknown = []

for key,value in data_count_by_year.items():
    #Female
    if key in dt_zeros_female:
        female.append(dt_zeros_female[key])
    else:
        female.append(0)

    #Male
    if key in dt_ones_male:
        male.append(dt_ones_male[key])
    else:
        male.append(0)

    #Unknown
    if key in dt_twos_unknown:
        unknown.append(dt_twos_unknown[key])
    else:
        unknown.append(0)
    years.append(key)
    total_entries.append(value)


In [27]:
d = {'Years' : years,
    'Male' : male,
    'Female' : female,
    'Unknown' : unknown,
    'Total_entries' : total_entries}
final_df = pd.DataFrame(d)

In [28]:
def percentage(part, whole):
    percentage = 100 * float(part)/float(whole)
    return round(percentage,2)

# print(percentage(3, 5))

In [29]:
male_percent_list = []
female_percent_list = []
unknown_percent_list = []
for index, row in final_df.iterrows():
    male_percent = percentage(row['Male'], row['Total_entries'])
    female_percent = percentage(row['Female'], row['Total_entries'])
    unknown_percent = percentage(row['Unknown'], row['Total_entries'])
    
    male_percent_list.append(male_percent)
    female_percent_list.append(female_percent)
    unknown_percent_list.append(unknown_percent)

In [30]:
final_df['Male_percentage'] = male_percent_list
final_df['Female_percentage'] = female_percent_list
final_df['Unknown_percentage'] = unknown_percent_list

# Q2

In [31]:
dt_symptoms = df['SYM_One Row Coded Symptoms']
dt_symptoms

0        SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL...
1        SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL...
2                             NAUSEA, DYSGEUSIA, DIARRHOEA
3                      GASTROINTESTINAL DISORDER, VOMITING
4          GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION
                               ...                        
90781                                             SWELLING
90782                                             SWELLING
90783                                             SWELLING
90784                            VOMITING, URTICARIA, RASH
90785                            VOMITING, URTICARIA, RASH
Name: SYM_One Row Coded Symptoms, Length: 90786, dtype: object

In [32]:
dt_symptoms

0        SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL...
1        SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL...
2                             NAUSEA, DYSGEUSIA, DIARRHOEA
3                      GASTROINTESTINAL DISORDER, VOMITING
4          GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION
                               ...                        
90781                                             SWELLING
90782                                             SWELLING
90783                                             SWELLING
90784                            VOMITING, URTICARIA, RASH
90785                            VOMITING, URTICARIA, RASH
Name: SYM_One Row Coded Symptoms, Length: 90786, dtype: object

In [33]:
symptoms_list  = []
for i in range(0,len(dt_symptoms)):
    temp = str(dt_symptoms[i])
    temp = temp.split(',')
    
    for j in range(0,len(temp)):
        txt = temp[j].strip()
        temp[j] = txt
    symptoms_list.extend(temp)

In [34]:
symptoms_count = dict(Counter(symptoms_list))
# print(symptoms_count)

In [35]:
q2_final_df = pd.DataFrame([symptoms_count])

In [36]:
q2_final_df = q2_final_df.transpose()

In [37]:
q2_final_df.rename(columns={0: "values"},inplace=True)

In [38]:
q2_final_df.sort_values('values',ascending=False,inplace=True)

In [39]:
# fig = px.bar(q2_final_df.head(50), x=q2_final_df.index[1:51], y='values')
# fig.show()

# Plotly

In [40]:
#final_df.head(10)

In [41]:
# import plotly.graph_objects as go

# fig = go.Figure(data=[
#     go.Bar(name='Female', x=final_df.Years, y=final_df.Female_percentage),
#     go.Bar(name='Male', x=final_df.Years, y=final_df.Male_percentage),
#     go.Bar(name='Unknown', x=final_df.Years, y=final_df.Unknown_percentage)],
#     layout=go.Layout(
#         title=go.layout.Title(text="Percent of Adverse Events Reported Each Year by Gender")))

# # Change the bar mode
# fig.update_layout(barmode='group')
# fig.show()

In [42]:
# Count_list = [final_df.Female.sum(),final_df.Male.sum(),final_df.Unknown.sum()]
# Name = ['Female','Male','Unknown']
# Count_list

In [43]:
# import plotly.express as px
# fig = px.pie(final_df,names=Name,values=Count_list,title='Percent of Adverse Events Reported all over the Years by Gender')
# fig.show()

In [44]:
# fig = px.bar(q2_final_df.head(50), x=q2_final_df.index[1:51], y='values')

In [45]:
# app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app = dash.Dash(__name__)

In [ ]:
BS = "https://cdn.jsdelivr.net/npm/bootstrap@5.1.0/dist/css/bootstrap.min.css"
app = dash.Dash(__name__,external_stylesheets=[BS])
app.layout = html.Div([
    html.H2(
        className='q1Header',
        children='Q1. Percent of Adverse Events Reported Each Year by Gender'),
    html.Div([
        html.Table([
            html.Tr([
                html.Td(html.P(className='From', children='From : ')),
                html.Td(
                    dcc.Dropdown(className='from_year_dropdown',
                                 id='from-year-dropdown',
                                 options=[{
                                     'label': year,
                                     'value': year
                                 } for year in final_df.Years],
                                 value=final_df.Years.max() - 10,
                                 clearable=False)),
                html.Td(html.P(className='To', children='To : ')),
                html.Td(
                    dcc.Dropdown(className='to_year_dropdown',
                                 id='to-year-dropdown',
                                 options=[{
                                     'label': year,
                                     'value': year
                                 } for year in final_df.Years],
                                 value=final_df.Years.max(),
                                 clearable=False))
            ])
        ])
    ]),
    html.Div([
        html.Div(className="q1_bar_chart",
                 children=[dcc.Graph(id='q1_bar_graph')]),
        html.Div(className="q1_pie_chart",
                 children=[dcc.Graph(id='q1_pie_graph')])
    ]),
    html.Div([html.Hr(className='Q_hr')]),
    html.Div([
        html.H2(
            className='q2Header',
            children='Q2. Most frequently Reported Symptoms over entire span'),
        html.Div(className='q2_graph',
                 children=[dcc.Graph(id='q2_bar_chart')]),
        html.Div(className='q2_graph_filters',
                 children=[
                     html.Div([html.P(className='filter',
                                      children='Filters')]),
                     html.Div([
                     html.Div(className='start',
                              children=[
                                  dcc.Input(id='start',
                                            type='number',
                                            placeholder='Eg.1',
                                            value=1)
                              ]),
                     html.Div(className='end',
                              children=[
                                  dcc.Input(id='end',
                                            type='number',
                                            placeholder='Eg.10',
                                            value=10)
                              ])
                     ]),
                     html.Br(),
                     html.Div(className = 'sort_head',
                              children = 'Sort'),
                     html.Div(className='q2_radio',
                              children=[
                                  dcc.RadioItems(id = 'q2_radio',
                                                 options=[{
                                      'label': 'High To Low',
                                      'value': 'high2low'
                                  }, {
                                      'label': 'Low To High',
                                      'value': 'low2high'
                                  }],
                                    value='high2low')
                              ])
                 ])
    ])
])


@app.callback(
    [Output('q1_bar_graph', 'figure'),
     Output('q1_pie_graph', 'figure')],
    [Input('from-year-dropdown', 'value'),
     Input('to-year-dropdown', 'value')])
def update_q1_bar_pie_chart(from_value, to_value):
    
    if from_value > to_value:
        raise PreventUpdate
    
    colors = ['rgb(239, 85, 59)', 'rgb(99, 110, 250)', 'rgb(0, 204, 150)']

    temp_df = final_df[(final_df.Years >= int(from_value))
                       & (final_df.Years <= int(to_value))]

    #Bar Chart
    fig_bar = go.Figure(data=[
        go.Bar(name='Male', x=temp_df.Years, y=temp_df.Male_percentage),
        go.Bar(name='Female', x=temp_df.Years, y=temp_df.Female_percentage),
        go.Bar(name='Unknown', x=temp_df.Years, y=temp_df.Unknown_percentage)
    ])

    fig_bar.update_layout(barmode='group')
    fig_bar.update_layout(showlegend=False)

    #Pie Chart
    Count_list = [
        temp_df.Male.sum(),
        temp_df.Female.sum(),
        temp_df.Unknown.sum()
    ]
    Name = ['Male', 'Female', 'Unknown']
    fig_pie = px.pie(final_df,
                     names=Name,
                     values=Count_list,
                     color_discrete_sequence=colors)

    fig_pie.update_layout(legend_title_text='Gender')

    return fig_bar, fig_pie


@app.callback(
    Output('q2_bar_chart', 'figure'),
    [Input('start', 'value'),
     Input('end', 'value'),
     Input('q2_radio','value')])
def Update_q2_bar_chart(start, end, sort):
    if (start is None) | (end is None) | (start > end):
        raise PreventUpdate
        
    slice_df = q2_final_df[start:end + 1]
    if(sort == 'low2high'):
        slice_df.sort_values('values', ascending=False, inplace=True)
    if(sort == 'high2low'):
        slice_df.sort_values('values', ascending=True, inplace=True)

    fig_bar = px.bar(slice_df,
                     x='values',
                     y=slice_df.index,
                    orientation='h')
    return fig_bar


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port='1234')

Dash is running on http://127.0.0.1:1234/

Dash is running on http://127.0.0.1:1234/

Dash is running on http://127.0.0.1:1234/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


<ipython-input-47-6cf797bc3e2e>:141: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-6cf797bc3e2e>:139: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

